<a href="https://colab.research.google.com/github/tabesink/AIE5/blob/main/AIE5_Midterm_Finetuning_Snowflake_Arctic_Embedding_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls "/content/drive/MyDrive/MgAlloy_Finetune_Dataset"

collections			      mg_alloy_finetuned_arctic_embed_l_ft_v1  val_dataset.jsonl
contextual_chunks		      test_dataset.jsonl
mg_alloy_finetuned_arctic_embed_l_ft  training_dataset.jsonl


In [3]:
#!pip install -qU langchain_openai langchain_huggingface langchain_core langchain langchain_community qdrant_client langchain_qdrant sentence_transformers fastembed

In [4]:
!pip install -qU sentence_transformers datasets pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key: ")

Enter Your OpenAI API Key: ··········


In [ ]:
CONTEXTUAL_CHUNKS_FOLDER="/content/drive/MyDrive/MgAlloy_Finetune_Dataset/contextual_chunks"
COLLECTIONS_PATH="/content/drive/MyDrive/MgAlloy_Finetune_Dataset/collections"

## Load Test Data

In [ ]:
import pickle
import tqdm
from tqdm import tqdm
from langchain_core.documents import Document

def load_chunks_as_documents(path):

    file_list = []
    if os.path.isfile(path) and path.endswith('.pkl'):
        # Single pkl file
        file_list.append(path)
    elif os.path.isdir(path):
        # Directory of pkl files
        for filename in os.listdir(path):
            if filename.endswith('.pkl'):
                path_ = os.path.join(path, filename)
                file_list.append(path_)

    loaded_chunk_data = {}
    for file in file_list:
        with open(file, 'rb') as f:
            data = pickle.load(f)
            loaded_chunk_data[data["filename"]] = data["chunks"]

    print(f"Loaded {len(loaded_chunk_data)} documents from {path}:")
    for i, doc_name in enumerate(loaded_chunk_data.keys()):
        print(f"  {i+1}. {doc_name}")

    # Convert the chunks to langhcain documents
    documents = []
    for fname in tqdm(loaded_chunk_data.keys(), desc="Converting chunks to documents"):

        chunks = loaded_chunk_data[fname]
        for chunk in chunks:
            documents.append(
                Document(
                    page_content=chunk.page_content,
                    metadata=chunk.metadata
                )
            )
    return documents


In [ ]:
training_documents = load_chunks_as_documents(CONTEXTUAL_CHUNKS_FOLDER)

Loaded 6 documents from /content/drive/MyDrive/MgAlloy_Finetune_Dataset/contextual_chunks:
  1. Fatigue_behaviour_and_fractography_of_extruded_AZ80
  2. Characterization_of_forged_magnesium_alloys
  3. Application_of_Machine_Learning_Modeling_in_Establishing_the_Process_Structure_and_Property_Relationships_of_the_Cast-Forged_AZ80_Magnesium_Alloy
  4. A_Method_for_Comparing_the_Fatigue_Performance_of_Forged
  5. Microstructure_and_texture_evolution_during_hot_deformation_of_AZ80_magnesium_alloy
  6. Fatigue_of_Forged_AZ80_Magnesium_Alloy


Converting chunks to documents: 100%|██████████| 6/6 [00:00<00:00, 2252.98it/s]


In [ ]:
print(f"Number of documents {len(training_documents)}")

Number of documents 638


In [ ]:
import uuid

id_set = set()

for document in training_documents:
  id = str(uuid.uuid4())
  while id in id_set:
    id = uuid.uuid4()
  id_set.add(id)
  document.metadata["id"] = id

In [ ]:
training_split_documents = training_documents[:len(training_documents) - 192]
val_split_documents = training_documents[len(training_documents) - 192:638-94]
test_split_documents = training_documents[638-94:]

In [ ]:
print(f"Number of training documents {len(training_split_documents)}")
print(f"Number of validation documents {len(val_split_documents)}")
print(f"Number of testing documents {len(test_split_documents)}")

Number of training documents 446
Number of validation documents 98
Number of testing documents 94


## Constructing a Fine-tuning Dataset

In [ ]:
from langchain_openai import ChatOpenAI

qa_chat_model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

qa_prompt = """\
Given the following context, you must generate questions based on only the provided context.

You are to generate {n_questions} questions which should be provided in the following format:

1. QUESTION #1
2. QUESTION #2
...

Context:
{context}
"""

qa_prompt_template = ChatPromptTemplate.from_template(qa_prompt)

In [ ]:
question_generation_chain = qa_prompt_template | qa_chat_model

In [ ]:
import tqdm

async def create_questions(documents, n_questions):
  questions = {}
  relevant_docs = {}

  ### YOUR CODE HERE
  id_set = set()
  for document in tqdm.tqdm(documents, desc="Processing documents"):
    # Generate question IDs and questions for each document
    for _ in range(n_questions):
      question_id = str(uuid.uuid4())
      while question_id in id_set:
        question_id = str(uuid.uuid4())
      id_set.add(question_id)

      # Generate a question based on document content
      questions[question_id] = await question_generation_chain.ainvoke({"context": document.page_content, "n_questions": n_questions})

      # Associate question with document ID
      relevant_docs[question_id] = [document.metadata["id"]]

  return questions, relevant_docs

In [ ]:
training_questions, training_relevant_contexts = await create_questions(training_split_documents, 2)

Processing documents: 100%|██████████| 446/446 [22:45<00:00,  3.06s/it]


In [ ]:
import json
# save datasets to gdrive
dataset_path = "/content/drive/MyDrive/MgAlloy_Finetune_Dataset"

In [ ]:
training_corpus = {train_item.metadata["id"] : train_item.page_content for train_item in training_split_documents}

training_questions = {k: str(v) for k, v in training_questions.items()}

train_dataset = {
    "questions" : training_questions,
    "relevant_contexts" : training_relevant_contexts,
    "corpus" : training_corpus
}

with open(os.path.join(dataset_path,"training_dataset.jsonl"), "w") as f:
  json.dump(train_dataset, f)

In [ ]:
val_questions, val_relevant_contexts = await create_questions(val_split_documents, 2)

Processing documents: 100%|██████████| 98/98 [06:05<00:00,  3.73s/it]


In [ ]:
val_corpus = {val_item.metadata["id"] : val_item.page_content for val_item in val_split_documents}

val_questions = {k: str(v) for k, v in val_questions.items()}

val_dataset = {
    "questions" : val_questions,
    "relevant_contexts" : val_relevant_contexts,
    "corpus" : val_corpus
}

with open(os.path.join(dataset_path,"val_dataset.jsonl"), "w") as f:
  json.dump(val_dataset, f)

In [ ]:
test_questions, test_relevant_contexts = await create_questions(test_split_documents, 2)

Processing documents: 100%|██████████| 94/94 [06:34<00:00,  4.20s/it]


In [ ]:
test_corpus = {test_item.metadata["id"] : test_item.page_content for test_item in test_split_documents}

test_questions = {k: str(v) for k, v in test_questions.items()}

test_dataset = {
    "questions" : test_questions,
    "relevant_contexts" : test_relevant_contexts,
    "corpus" : test_corpus
}

with open(os.path.join(dataset_path,"test_dataset.jsonl"), "w") as f:
  json.dump(test_dataset, f)

## Fine-tuning snowflake-arctic-embed-l

In [6]:
# load ft datasets
import os
import json
dataset_path = "/content/drive/MyDrive/MgAlloy_Finetune_Dataset"

with open(os.path.join(dataset_path,"training_dataset.jsonl"), "r") as f:
    train_dataset = json.load(f)

with open(os.path.join(dataset_path,"test_dataset.jsonl"), "r") as f:
    test_dataset = json.load(f)

with open(os.path.join(dataset_path,"val_dataset.jsonl"), "r") as f:
    val_dataset = json.load(f)

In [7]:
from sentence_transformers import SentenceTransformer

model_id = "Snowflake/snowflake-arctic-embed-l"
model = SentenceTransformer(model_id)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/85.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/107 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [9]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sentence_transformers import InputExample

In [10]:
BATCH_SIZE = 16

In [11]:
# move dataset into the expected format for training.
corpus = train_dataset['corpus']
queries = train_dataset['questions']
relevant_docs = train_dataset['relevant_contexts']

examples = []
for query_id, query in queries.items():
    doc_id = relevant_docs[query_id][0]
    text = corpus[doc_id]
    example = InputExample(texts=[query, text])
    examples.append(example)

In [12]:
loader = DataLoader(
    examples, batch_size=BATCH_SIZE
)

In [13]:
from sentence_transformers.losses import MatryoshkaLoss, MultipleNegativesRankingLoss

matryoshka_dimensions = [768, 512, 256, 128, 64]
inner_train_loss = MultipleNegativesRankingLoss(model)
train_loss = MatryoshkaLoss(
    model, inner_train_loss, matryoshka_dims=matryoshka_dimensions
)

In [14]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator

corpus = val_dataset['corpus']
queries = val_dataset['questions']
relevant_docs = val_dataset['relevant_contexts']

evaluator = InformationRetrievalEvaluator(queries, corpus, relevant_docs)

In [15]:
import wandb
wandb.init(mode="disabled")

In [16]:
#!pip install -qU faiss-cpu python-pptx==1.0.2 nltk==3.9.1 pymupdf beautifulsoup4 lxml

In [17]:
#!pip install sentence-transformers==3.3.1

In [18]:
#!pip install datasets==3.3.0

In [20]:
EPOCHS = 25

In [21]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sentence_transformers import InputExample

warmup_steps = int(len(loader) * EPOCHS * 0.1)

model.fit(
    train_objectives=[(loader, train_loss)],
    epochs=EPOCHS,
    warmup_steps=warmup_steps,
    output_path=os.path.join(dataset_path, 'mg_alloy_finetuned_arctic_embed_l_ft_v2'),
    show_progress_bar=True,
    evaluator=evaluator,
    evaluation_steps=50
)

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss,Cosine Accuracy@1,Cosine Accuracy@3,Cosine Accuracy@5,Cosine Accuracy@10,Cosine Precision@1,Cosine Precision@3,Cosine Precision@5,Cosine Precision@10,Cosine Recall@1,Cosine Recall@3,Cosine Recall@5,Cosine Recall@10,Cosine Ndcg@10,Cosine Mrr@10,Cosine Map@100
50,No log,No log,0.897959,0.979592,0.989796,0.989796,0.897959,0.326531,0.197959,0.098980,0.897959,0.979592,0.989796,0.989796,0.952522,0.939626,0.939751
56,No log,No log,0.928571,0.979592,0.989796,0.989796,0.928571,0.326531,0.197959,0.098980,0.928571,0.979592,0.989796,0.989796,0.963820,0.954932,0.955061
100,No log,No log,0.959184,0.989796,0.989796,1.000000,0.959184,0.329932,0.197959,0.100000,0.959184,0.989796,0.989796,1.000000,0.980111,0.973810,0.973810
112,No log,No log,0.943878,0.984694,0.989796,1.000000,0.943878,0.328231,0.197959,0.100000,0.943878,0.984694,0.989796,1.000000,0.974337,0.965914,0.965914
150,No log,No log,0.959184,1.000000,1.000000,1.000000,0.959184,0.333333,0.200000,0.100000,0.959184,1.000000,1.000000,1.000000,0.984936,0.979592,0.979592
168,No log,No log,0.948980,1.000000,1.000000,1.000000,0.948980,0.333333,0.200000,0.100000,0.948980,1.000000,1.000000,1.000000,0.981170,0.974490,0.974490
200,No log,No log,0.964286,1.000000,1.000000,1.000000,0.964286,0.333333,0.200000,0.100000,0.964286,1.000000,1.000000,1.000000,0.986819,0.982143,0.982143
224,No log,No log,0.989796,1.000000,1.000000,1.000000,0.989796,0.333333,0.200000,0.100000,0.989796,1.000000,1.000000,1.000000,0.996234,0.994898,0.994898
250,No log,No log,0.979592,1.000000,1.000000,1.000000,0.979592,0.333333,0.200000,0.100000,0.979592,1.000000,1.000000,1.000000,0.992468,0.989796,0.989796
280,No log,No log,0.984694,1.000000,1.000000,1.000000,0.984694,0.333333,0.200000,0.100000,0.984694,1.000000,1.000000,1.000000,0.994351,0.992347,0.992347


In [22]:
from huggingface_hub import notebook_login
notebook_login()

In [23]:
hf_username = "tabesink92"
model.push_to_hub(f"{hf_username}/mg_alloy-snowflake-arctic-embed-l-ft-v2")

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

'https://huggingface.co/tabesink92/mg_alloy-snowflake-arctic-embed-l-ft-v2/commit/ad03d4d13946948f4ff006256270f41be52fd664'

## Evaluating Retriever

In [ ]:
import pandas as pd

from langchain_community.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.documents import Document

In [ ]:
def evaluate_openai(
    dataset,
    embed_model,
    top_k=5,
    verbose=False,
):
  corpus = dataset['corpus']
  questions = dataset['questions']
  relevant_docs = dataset['relevant_contexts']
  documents = [Document(page_content=content, metadata={"id": doc_id}) for doc_id, content in corpus.items()]
  vectorstore = FAISS.from_documents(documents, embed_model)

  retriever = vectorstore.as_retriever(search_kwargs={"k": top_k})

  eval_results = []
  for id, question in tqdm(questions.items()):
    retrieved_nodes = retriever.invoke(question)
    retrieved_ids = [node.metadata["id"] for node in retrieved_nodes]
    expected_id = relevant_docs[id][0]
    is_hit = expected_id in retrieved_ids
    eval_results.append({"id": id, "question": question, "expected_id": expected_id, "is_hit": is_hit})

  return eval_results

In [ ]:
# Snowflake/snowflake-arctic-embed-l (base)
from langchain_huggingface import HuggingFaceEmbeddings

huggingface_embeddings = HuggingFaceEmbeddings(model_name="Snowflake/snowflake-arctic-embed-l")
arctic_embed_m_results = evaluate_openai(test_dataset, huggingface_embeddings)

In [ ]:
arctic_embed_m_results_df = pd.DataFrame(arctic_embed_m_results)

In [ ]:
arctic_embed_m_hit_rate = arctic_embed_m_results_df["is_hit"].mean()
arctic_embed_m_hit_rate

In [ ]:
# Snowflake/snowflake-arctic-embed-l (fine-tuned)
#os.path.join(dataset_path, 'mg_alloy_finetuned_arctic_embed_l_ft_v2')
finetune_embeddings = HuggingFaceEmbeddings(model_name="mg_alloy_finetuned_arctic_embed_l_ft_v2")
finetune_results = evaluate_openai(test_dataset, finetune_embeddings)
finetune_results_df = pd.DataFrame(finetune_results)
finetune_hit_rate = finetune_results_df["is_hit"].mean()
finetune_hit_rate

#### RAG - Base Chain

In [ ]:
from langchain_community.vectorstores import FAISS

base_vectorstore = FAISS.from_documents(training_documents, huggingface_embeddings)
base_retriever = base_vectorstore.as_retriever(search_kwargs={"k": 6})

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and a question, you must answer the question. If you do not know the answer, you must state that you do not know.

Context:
{context}

Question:
{question}

Answer:
"""

rag_prompt_template = ChatPromptTemplate.from_template(RAG_PROMPT)

In [ ]:
rag_llm =  ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

In [ ]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

base_rag_chain = (
    {"context": itemgetter("question") | base_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt_template | rag_llm | StrOutputParser(), "context": itemgetter("context")}
)

#### Fine-tuned Chain

In [ ]:
finetune_vectorstore = FAISS.from_documents(training_documents, finetune_embeddings)
finetune_retriever = finetune_vectorstore.as_retriever(search_kwargs={"k": 6})

In [ ]:
finetune_rag_chain = (
    {"context": itemgetter("question") | finetune_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt_template | rag_llm | StrOutputParser(), "context": itemgetter("context")}
)

### RAGAS Evaluation

In [ ]:
!pip install -qU ragas==0.2.10 rapidfuzz

In [ ]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from ragas.testset import TestsetGenerator

generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(training_documents, testset_size=10)

dataset.to_pandas()

In [ ]:
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.documents import Document

RAG_PROMPT = """\
You are a helpful assistant who answers questions based on provided context. You must only use the provided context, and cannot use your own knowledge.

### Question
{question}

### Context
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)


class State(TypedDict):
  question: str
  context: List[Document]
  response: str

def generate(state):
  docs_content = "\n\n".join(doc.page_content for doc in state["context"])
  messages = rag_prompt.format_messages(question=state["question"], context=docs_content)
  response = llm.invoke(messages)
  return {"response" : response.content}

def retrieve(state):
  #retrieved_docs = retriever.invoke(state["question"])
  retrieved_docs = base_retriever.invoke(state["question"])
  return {"context" : retrieved_docs}

graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
for test_row in dataset:
  response = graph.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]

In [ ]:
dataset.to_pandas()

### Base Retriever

In [ ]:
from ragas import EvaluationDataset
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper


evaluation_dataset = EvaluationDataset.from_pandas(dataset.to_pandas())
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))

In [ ]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
from ragas import evaluate, RunConfig

custom_run_config = RunConfig(timeout=360)

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
result

### FINE_TUNED RETRIEVER

In [ ]:
def retrieve(state):
  #retrieved_docs = retriever.invoke(state["question"])
  retrieved_docs = finetune_retriever.invoke(state["question"])
  return {"context" : retrieved_docs}


graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [ ]:
evaluation_dataset = EvaluationDataset.from_pandas(dataset.to_pandas())
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))

custom_run_config = RunConfig(timeout=360)

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
result